In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras_preprocessing.image import ImageDataGenerator


from src.ml.eval import eval_model
from ml.pipeline import FeitDataPipeline
from tensorflow import keras

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import schedules
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import concatenate

from cfg import *

from livelossplot import PlotLossesKerasTF

print(os.getcwd())
os.chdir('/home/jelinek/recetox/')

name = "MySimpleCNN_Inception_module-v2-reduced-no-inception-no-pooling-depthwise-proper"


/home/jelinek/recetox/src/notebooks/model_heaven


In [6]:
class MySimpleCNNInceptionModule(FeitDataPipeline):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        lr_schedule = schedules.ExponentialDecay(
            initial_learning_rate=1e-2,
            decay_steps=30,
            decay_rate=0.1,
            staircase=True)

        self.optimizer = Adam(
            learning_rate=lr_schedule,
            beta_1=0.99,
            beta_2=0.9999)

        self.model = self.get_compiled_model()
        self.params.name = name
        self.params.epochs = 200
        self.batch_size = 16
        self.params.tile_size=256


    @staticmethod
    def get_compiled_model():
        inputs = keras.Input(shape=(256, 256, 3))

        x = keras.layers.DepthwiseConv2D(depth_multiplier=2, kernel_size=5, strides=(2, 2), padding='same')(inputs)
        x = keras.layers.Conv2D(filters=6, kernel_size=1, strides=(1, 1), padding='same')(x)

        x = keras.layers.DepthwiseConv2D(depth_multiplier=2, kernel_size=5, strides=(2, 2), padding='same')(x)
        x = keras.layers.Conv2D(filters=12, kernel_size=1, strides=(1, 1), padding='same')(x)

        x = keras.layers.DepthwiseConv2D(depth_multiplier=2, kernel_size=5, strides=(2, 2), padding='same')(x)
        x = keras.layers.Conv2D(filters=24, kernel_size=1, strides=(1, 1), padding='same')(x)

        x = keras.layers.DepthwiseConv2D(depth_multiplier=2, kernel_size=5, strides=(2, 2), padding='same')(x)
        x = keras.layers.Conv2D(filters=48, kernel_size=1, strides=(1, 1), padding='same')(x)

        x = keras.layers.DepthwiseConv2D(depth_multiplier=2, kernel_size=5, strides=(2, 2), padding='same')(x)
        x = keras.layers.Conv2D(filters=96, kernel_size=1, strides=(1, 1), padding='same')(x)

        # x = keras.layers.DepthwiseConv2D(depth_multiplier=2, kernel_size=5, strides=(2, 2), padding='same')(x)
        # x = keras.layers.Conv2D(filters=192, kernel_size=1, strides=(1, 1), padding='same')(x)
        x = keras.layers.AveragePooling2D(padding='same', pool_size=(2, 2))(x)

        x = keras.layers.Flatten()(x)
        outputs = keras.layers.Dense(units=12, activation='softmax')(x)

        model = keras.Model(inputs, outputs, name='MySimpleCnnFewerLayers')
        model.summary()
        return model

    def get_data_loader_training(self):
        datagen_train = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, samplewise_center=True,
                                           samplewise_std_normalization=True)

        return datagen_train.flow_from_directory(directory=self.params.data_training, color_mode='rgb',
                                                 class_mode='categorical', batch_size=self.params.batch_size,
                                                 shuffle=True,
                                                 target_size=(self.params.tile_size, self.params.tile_size))

    def get_data_loader_validation(self):
        datagen_valid = ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)
        return datagen_valid.flow_from_directory(directory=self.params.data_validation, color_mode='rgb',
                                                 class_mode='categorical', batch_size=self.params.batch_size,
                                                 shuffle=False,
                                                 target_size=(self.params.tile_size, self.params.tile_size))


    def _train_model(self, data_train, data_valid):

        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                      patience=30, min_lr=1e-4, verbose=1,
                                      cooldown=20)

        self.model.fit(data_train,
                       steps_per_epoch=250,
                       epochs=200,
                       shuffle=True,
                       validation_data=data_valid,
                       validation_freq=10,
                       verbose=1,
                       callbacks=[self.tensorboard, reduce_lr, PlotLossesKerasTF()])

In [7]:
pipeline = MySimpleCNNInceptionModule(train_data_dir='data/Feit_colon-annotation-tiles-256/data_train/',
                        valid_data_dir='data/Feit_colon-annotation-tiles-256/data_valid/')

Model: "MySimpleCnnFewerLayers"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
depthwise_conv2d_6 (Depthwis (None, 128, 128, 6)       156       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 6)       42        
_________________________________________________________________
depthwise_conv2d_7 (Depthwis (None, 64, 64, 12)        312       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 64, 12)        156       
_________________________________________________________________
depthwise_conv2d_8 (Depthwis (None, 32, 32, 24)        624       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 

In [4]:
pipeline.execute_pipeline(perform_validation=True, perform_test_segmentation=False)

Found 28369 images belonging to 11 classes.
Found 1617 images belonging to 12 classes.
Model: "MySimpleCnnFewerLayers"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
depthwise_conv2d_6 (Depthwis (None, 128, 128, 6)       156       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 6)       42        
_________________________________________________________________
depthwise_conv2d_7 (Depthwis (None, 64, 64, 12)        312       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 64, 12)        156       
_________________________________________________________________
depthwise_conv2d_8 (Depthwis (None, 32, 32, 24)        624       
_______________________

KeyboardInterrupt: 

In [9]:
pipeline.save_pipeline()

In [5]:
eval_model(pipeline.model,
           pipeline.get_data_loader_validation(),
           print_confusion_matrix=True,
           save_misclassified=True)

Found 1617 images belonging to 12 classes.
102/102 [==============================] - 5s 53ms/step - loss: 0.9717 - categorical_accuracy: 0.6543
Measuring time
--Iteration 10/10
Batch size: 16
Mean time per tile 0.4119ms
Confusion Matrix
[[236   2  15   0  15   0   3   0   0   0  64   0]
 [  5  32  16   0   2   0   4   2   0   0   3   0]
 [ 23  13  49   0   7   0  22   5   0   0  26   0]
 [  0   0   0  11   0   0   0   0   0   0   0   0]
 [  0   0   1   3 537   0   0   0   0   0   3   0]
 [  0   0   1   0   0   0   0   0   0   0   0   0]
 [ 31   8  11   0   0   0   7  18   0   0   2   0]
 [  3   0   0   0   0   0   6  77   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   1   0]
 [116   0   2   0   7   0   1   0   0   0 108   0]
 [  6   0   3   0  33   0   0   0   0   0  75   1]]
Classification Report
                             precision    recall  f1-score   support

             adenocarcinoma       0.56      0.70      0.6

In [2]:
pipeline = FeitDataPipeline.load_pipeline(pipeline_name=name)